To do: 
- Update driver_locs to have anonymized coordinates only 
- change 'ahm' to 'cityA' everywhere
- Provide clear instructions on:
    - how to generate the assignments and get the de_intervals file on each day 
    - what changes are needed in the FoodMatch data directory to accomodate the new assignements i.e., where to put the newly generated de_intervals
  

In [ ]:
# Imports 
# IMPORTING STUFF

import os 
import csv
import math
import time
import copy
import glob 
import pickle
import shutil
import random
import ortools                       
import logging
import datetime
import matplotlib 
import numpy as np
import pandas as pd
import configparser  
import plotly.express as px  
from shapely import geometry
from functools import reduce
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim  

matplotlib.rc('xtick', labelsize=26) 
matplotlib.rc('ytick', labelsize=26) 

plt.rcParams['font.size'] = '26'
plt.rcParams['figure.figsize'] = (10,7.5)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50

In [ ]:
city = 'ahm'

flag = 0
w1, w2 = 0.5, 0.5
'''
"flag" decides which distance metric/measure to consider:
0: euclidean distance (or physical distance)
1: rating
2: combination of euclidean distance and rating
   where 'w1' is weight given to euclidean distance and 'w2' is weight given to rating
'''

In [ ]:
# LOADING DATASETS
import glob 

driver_files_ahm = sorted(glob.glob("data/driver_locs/ahm/driver_data_ahm_day*.csv"))
num_days = len(driver_files_ahm) 
driver_dfs_ahm = [pd.read_csv(driver_files_ahm[idx]) for idx in range(num_days)]
driver_dfs_dict = {'ahm': driver_dfs_ahm}
zone_df_ahm = pd.read_csv("data/zone_data/zone_data_ahm.csv")
zone_dfs_dict = {'ahm': zone_df_ahm}
income_df_ahm = pd.read_csv("data/income_data/incomes_ahm.csv")
income_dfs_dict = {'ahm': income_df_ahm} 
base_zone_ahm = pd.read_csv("data/base_zones/ahm_base_zones.csv")
base_zones_dict = {'ahm': base_zone_ahm}

In [ ]:
# UTILITIES
#'city' takes values in {'ahm', 'blr', 'del'}

def driver_union(drivers_dict):
    """
    Finding union of all the drivers over the days 
    """
    driver_dfs = drivers_dict[city] 
    num_days = len(driver_dfs)
    
    driver_udf = pd.concat([driver_dfs[idx] for idx in range(num_days)])
    driver_udf = driver_udf.drop('Unnamed: 0', axis=1)
    driver_udf = driver_udf.drop_duplicates('de_id').reset_index().drop('index', axis=1)
    
    return driver_udf


def driver_intersection(drivers_dict):
    """
    Finding intersection of all the drivers over the days
    """
    driver_dfs = drivers_dict[city]
    driver_idf = reduce(lambda left,right: pd.merge(left,right,on='de_id'), driver_dfs)
    driver_idf = driver_idf[['de_id', 'lat_x', 'lng_x']]
    driver_idf = driver_idf.loc[:, ~driver_idf.columns.duplicated()] 
    driver_idf = driver_idf.rename(columns={'lat_x':'lat', 'lng_x':'lng'})
    
    return driver_idf



def drivers_zones(drivers_dict, zones_dict):
    """
    To get the data to be input to fair_clustering: "driver_locs" and "zone_locs"
    """
    driver_idf = driver_intersection(drivers_dict) 
    
    # finding "driver_locs":
    driver_locs = driver_idf[['lat', 'lng']] 
    driver_locs = driver_locs.values 
    
    # finding "zone_locs":
    zone_df = zones_dict[city]
    zone_locs = np.array(zone_df[['lat', 'lng']])
    
    return driver_locs, zone_locs


def get_capacities(zones_dict):
    """
    returns "lower_caps" and "upper_caps"
    lower_caps: [1 x num_centres] array with lower capacity of each zone
    upper_caps: [1 x num_centres] array with upper capacity of each zone
    """
    zone_df = zones_dict[city] 

    lower_caps = 0.3*zone_df['avg_cap'].values
    upper_caps = 1.0*zone_df['avg_cap'].values
    
    return lower_caps, upper_caps

In [ ]:
# LOGGER

import logging
logging.basicConfig(filename=city+".log", format='%(asctime)s  %(message)s', filemode='w')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

**Getting the inputs to fair_clustering**

In [ ]:
driver_udf = driver_union(driver_dfs_dict)
driver_idf = driver_intersection(driver_dfs_dict) 

driver_locs, zone_locs = drivers_zones(driver_dfs_dict, zone_dfs_dict)
num_drivers, num_centres = driver_locs.shape[0], zone_locs.shape[0]

lower_caps, upper_caps = get_capacities(zone_dfs_dict)
assert driver_locs.shape[0]>lower_caps.sum() and driver_locs.shape[0]<upper_caps.sum(), \
"This set of num_drivers, lower_caps and upper_caps will lead to an infeasible solution !"

In [ ]:
def euclidean_distance(d_loc, z_loc):
    lat1, lng1 = d_loc[0], d_loc[1]
    lat2, lng2 = z_loc[0], z_loc[1]

    dist = np.sqrt(np.power(lat1-lat2, 2) + np.power(lng1-lng2, 2))
    return dist

def L2Distance(data):
  # "data": latitude-longitude level locations 
  transposed = np.expand_dims(data, axis = 1)
  distance = np.power(data - transposed, 2)
  distance = np.power(np.abs(distance).sum(axis = 2), 0.5) 
  return distance 

driver_dists = L2Distance(driver_locs)

In [ ]:
num_drivers, num_zones = driver_locs.shape[0], zone_locs.shape[0]

dz_dist = np.zeros(shape=(num_drivers, num_zones))
for d_idx, driver in enumerate(driver_locs):
    d_dist = np.zeros(num_zones)
    for z_idx, zone in enumerate(zone_locs):
        dist = euclidean_distance(driver, zone)
        d_dist[z_idx] = dist 
    dz_dist[d_idx] = d_dist

dz_dist.shape

In [ ]:
def get_pa(d_dist, k):
    prohibited_assignments = np.zeros(shape=(num_drivers, num_zones))
    
    for d_idx, d_dist in enumerate(dz_dist):
        idx = np.argpartition(d_dist, k) 
        prohibited_assignments[d_idx][idx[k:]] = 1 # set the indices NOT corresponding to k-smallest elements 
    
    return prohibited_assignments

In [ ]:
# assigning ratings to sellers:
from scipy.stats import truncnorm
from numpy.random import SeedSequence 
from numpy.random import default_rng

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm( (low-mean)/sd, (upp-mean)/sd, loc=mean, scale=sd) 

def generate_ratings(num_drivers):
    mean = 3.5
    sd = 1
    min_rating = 0.0
    max_rating = 5.0
    seedVal = 36778738061272522495168595294022739449 # arbitrary
    rng = default_rng(seedVal)
    dist = get_truncated_normal(mean, sd, min_rating, max_rating)
    ratings = dist.rvs(num_drivers, random_state=rng)
    ratings = [round(x, 1) for x in ratings]
    return ratings

def abs_difference(ratings):
    transposed = np.expand_dims(ratings, axis=1)
    diff = abs(ratings-transposed) 
    return diff   

def minmax(distance, fair_distance):
    num_samples = len(distance)
    mx, mn = distance.max(), distance.min()
    dists = distance.flatten()
    dists = np.asarray( [((x-mn)/(mx-mn)) for x in dists] )
    distance = dists.reshape((num_samples, num_samples))
    fair_distance = (fair_distance-mn)/(mx-mn)
    return distance, fair_distance

In [ ]:
zone_ids = zone_dfs_dict[city]['zone_id']
zone_id2idx = {zone_id: idx for idx, zone_id in enumerate(zone_ids)}

In [ ]:
location_df = pd.read_csv(f"data/location_data/location_{city}.csv", header=None)
location_df = location_df.rename(columns={0:'node_id', 1:'lat', 2:'lng'})
location_df.head()

In [ ]:
# Random generation of locations within a given zone

# generating random locations withing a zone (given the zone boundary):
def random_loc_generator(zone_bdry):
  lats, longs = path_related_preprocessing(zone_bdry)
  coords = [(x,y) for x,y in zip(lats, longs)]
  min_lat, max_lat = min(lats), max(lats)
  min_lng, max_lng = min(longs), max(longs)
  new_lat, new_lng = random.uniform(min_lat, max_lat), random.uniform(min_lng, max_lng) 
  return [new_lat, new_lng]

# Checking if a given location lies inside a given zone:
def path_related_preprocessing(path_bdry):
  # exemplar path_bdry: '12.954619258010608,77.6149292592163 12.954680993923494,77.61640664016727 ....'
  path_bdry = str(path_bdry)
  df = pd.DataFrame({'lts':[], 'lngs':[]})
  bdry_locs = path_bdry.split()
  lats, longs = [], []
  for loc in bdry_locs:
    lat, lng = loc.split(',')
    lats.append(float(lat))
    longs.append(float(lng))
  return lats, longs

def loc_in_zone(loc, zone_bdry):
  lats, longs = path_related_preprocessing(zone_bdry)
  coords = [(x,y) for x,y in zip(lats, longs)]
  polygon = geometry.MultiPoint(coords).convex_hull
  Point_X, Point_Y = loc[0], loc[1]
  point = geometry.Point(Point_X, Point_Y)
  return point.within(polygon)

# code to generate 'm' locations that lie within a given zone:
def generate_locs(m, zone_bdry):
    new_locs = []
    num_generated = 0
    while num_generated < m:
        new_loc = random_loc_generator(zone_bdry)
        sanity_check = loc_in_zone(new_loc, zone_bdry)
        if sanity_check:
            num_generated += 1
            new_locs.append(new_loc)
    return new_locs

---

**ALGORITHMS**

**FairAssign**

In [ ]:
# will go with the default parameters of cplex:
from cplex import Cplex
model = Cplex()
model.parameters.simplex.tolerances.feasibility.get(),\
model.parameters.simplex.tolerances.optimality.get(),\
model.parameters.simplex.tolerances.markowitz.get()      

model.parameters.workmem.set(10240) # 10GB  
model.parameters.emphasis.memory.set(1)

In [ ]:
# Fair Clustering - LPP contstraints and Cplex
from cplex import Cplex
# from lp_tools import *
from lp_tools_kn import * 

alpha_fair = 2

def fair_clustering(dataset, centres, lower_cap, upper_cap, fair_distance, prohibited_assignments):
  # Step 1: 	 Create an instance of Cplex 
  problem = Cplex()
  problem.parameters.simplex.tolerances.feasibility.set(float(1e-9))
  problem.parameters.simplex.tolerances.optimality.set(float(1e-9))
  problem.parameters.simplex.tolerances.markowitz.set(float(0.9999)) 
  problem.parameters.emphasis.memory.set(1)
  problem.parameters.workmem.set(10240)

  # Step 2: 	 Declare that this is a minimization problem
  problem.objective.set_sense(problem.objective.sense.minimize)
    
  """
   Step 3.   Declare and  add variables to the model. 
        The function prepare_to_add_variables (dataset, centres) prepares all the required information for this stage.
  
    objective: a list of coefficients (float) in the linear objective function
    lower bound: a list of floats containing the lower bounds for each variable
    upper bound: a list of floats containing the upper bounds for each variable
    variable_names: a list of strings that contains the name of the variables
  """
  ## if working with "lp_tools":
  print("Adding Variables...")
  
  # objective, lower_bound, upper_bound, variable_names, P,C = prepare_to_add_variables(dataset, centres)
  ## if working with "lp_tools_kn": 
  objective, lower_bound, upper_bound, variable_names, P,C = prepare_to_add_variables(dataset, centres, prohibited_assignments)
  problem.variables.add(
      obj = objective,
      lb = lower_bound,
      ub = upper_bound,
      names = variable_names
     
    )
  
  print("Variables Added !")
    
    
  """
  Step 4.   Declare and add constraints to the model.
            There are few ways of adding constraints: row wise, col wise and non-zero entry wise.
            Assume the constraint matrix is A. We add the constraints non-zero entry wise.
            The function prepare_to_add_constraints(dataset, centres) prepares the required data for this step.
  
   coefficients: Three tuple containing the row number, column number and the value of the constraint matrix
   senses: a list of strings that identifies whether the corresponding constraint is
           an equality or inequality. "E" : equals to (=), "L" : less than (<=), "G" : greater than equals (>=)
   rhs: a list of floats corresponding to the rhs of the constraints.
   constraint_names: a list of string corresponding to the name of the constraint
  """
  print("Adding Constraints...")
    
  rhs, senses, row_names, coefficients = prepare_to_add_constraints(dataset, centres, upper_cap,lower_cap, P,C, alpha_fair, fair_distance, ratings, flag)
  print("num_constraints:", len(senses)) 
  logger.info(f"\t\t\tnum_constraints = {len(senses)}")
  problem.linear_constraints.add(
      rhs = rhs,
      senses = senses,
      names = row_names
    )
  problem.linear_constraints.set_coefficients(coefficients)

  print("Constraints Added !")
    
  # Step 5.	Solve the problem
  problem.solve()

  result = {
    "status": problem.solution.get_status(),
    "success": problem.solution.get_status_string(),
    "objective": problem.solution.get_objective_value(),
    "assignment": problem.solution.get_values(),
  }
    
  qm = problem.solution.quality_metric  
  print("Solution Quality:", problem.solution.get_float_quality([qm.max_x, qm.max_primal_infeasibility]))
  
  # print("Status:", result['status']) # outputs a number: "1" for optimal solution, "2" for unbounded ray and "3" for infeasible solution
  solution_status = result['status']
  assert solution_status==1, "Solution isn't optimal !"

  print("Status:", problem.solution.get_status_string()) # optimal, unbounded ray, infeasible

  return result
'''

In [ ]:
# Fair Assignment of drivers to the FFCs / warehouses
import copy
import dependent_routing as dp

# configParser.read(configFilePath)

num_samples, num_centres = driver_locs.shape[0], zone_locs.shape[0]

def fair_assignment(prob_dis,driver_loc):
  '''Assigning the driver using the probaility distribution using dependent rounding'''  
  
  # "prob_dis" is the result of the Fair-LP program "fair_clustering"  
  prob_dist = copy.deepcopy(prob_dis)
  # print("prob_dist shape [num_drivers x num_ffc]:", prob_dist.shape)

  rounding = dp.DependentRounding(prob_dist)
  rounding._buildGraph(prob_dist)
  final_assignment = rounding.round()
  final_assignment = np.around(final_assignment,2)

  driver_df = pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df['ffc_index'] = -1 # unassigned

  for i in range(num_samples):
    for j in range(num_centres):
      if abs(final_assignment[i][j]-1) < 0.01: 
        driver_df.at[i,'ffc_index'] = j
        
  return driver_df, final_assignment


In [ ]:
def sanityCheck(probs):
    """
    To cope with bound violations which can occur upto the feasibility parameter range 
    So the lower bound of 0.0 on the probabilities can get violated and the values can go down to (0-feasibility_parameter_value)
    """
    for i in range(len(probs)):
        last_pos_index = -1
        neg_value = 0
        
        for j in range(len(probs[0])):
            assert probs[i][j] >= -1e-6 
            
            if probs[i][j] < 0:
                neg_value += probs[i][j]
                probs[i][j] = 0
            elif probs[i][j] > 0:
                last_pos_index = j

        max_pos_index = np.argmax(probs[i])
        probs[i][max_pos_index] += neg_value
        
        assert probs[i][max_pos_index] > 0
        
    return probs

In [ ]:
# main :
def FairAssign_solver(driver_locs, zone_locs, lower_cap, upper_cap, fair_distance, prohibited_assignments):
    # Fair-LP:
    try:
        lp_output = fair_clustering(driver_locs, zone_locs, lower_cap, upper_cap, fair_distance, prohibited_assignments)
    except:
        logger.error("Solution Non-optimal (Unbounded Ray or Infeasible) !")
        return None, None
    prob_dis = np.reshape(lp_output['assignment'][:num_samples*num_centres], (-1, num_centres))
    
    try:
        prob_dist = sanityCheck(copy.deepcopy(prob_dis)) # this might raise an assertion error
    except:
        logger.error("Sanity Check Assertion !")
        return None, None
    
    # Randomized Dependent Rounding:
    try:
        df = fair_assignment(prob_dist, driver_locs)[0] # this might raise an assertion error
        final_assignment = df['ffc_index'].values
    except:
        logger.error("Dependent Rounding Assertion !")
        return prob_dist, None
    
    return prob_dist, final_assignment
    

In [ ]:
k_list = [(x-1) for x in [7, 5, 3]] 
fd_list = [(driver_dists.mean()/alpha) for alpha in [28, 14, 8]] 
num_runs = 5
# k_list and fd_list contain hyperparameters

k_fd_dict = {k:\
                {fd_idx:\
                    {num_run:\
                        {'p_dist':None, 'assignment':None}
                        for num_run in range(num_runs)
                    } 
                    for fd_idx in range(len(fd_list))
                } 
            for k in k_list
            }

os.mkdir("./assign_results") # directory to store the results of FairAssign_solver

for k in k_list:
    print(k)
    logger.info(f"Considering k = {k+1} nearest zones")
    prhbtd_assigns = get_pa(dz_dist, k)
    
    for f_idx, fair_distance in enumerate(fd_list):
        logger.info(f"\tfair_distance = {fair_distance}")
        for num_run in range(num_runs):
            logger.info(f"\t\tnum_run = {num_run}")
            prob_dist, final_assignment = FairAssign_solver(driver_locs, zone_locs, lower_caps, upper_caps, fair_distance, prhbtd_assigns)
            k_fd_dict[k][f_idx][num_run]['p_dist'] = prob_dist
            k_fd_dict[k][f_idx][num_run]['assignment'] = final_assignment 
    # Store intermediate results as well as fail-safe:
    # saving current state of "k_fd_dict":
    pickling_on = open(f"dict_k={k+1}.pickle", "wb")
    pickle.dump(k_fd_dict, pickling_on)
    pickling_on.close() 

pickling_on = open("Assignments_ratings_"+city+".pickle", "wb")
pickle.dump(k_fd_dict, pickling_on)
pickling_on.close()

In [ ]:
def get_assignment(result_path, driver_locs, k):
    '''
    returns ffc_index (or zone index) for each driver based on the 
    '''
    pickle_off = open(result_path, "rb")
    assignments = pickle.load(pickle_off)
    prob_dist = assignments[k-1][0][0]['p_dist']
    ## get assignment by applying dependent rounding: 
    df = fair_assignment(prob_dist, driver_locs)[0] 
    final_assignment = df['ffc_index'].values

    return final_assignment

In [ ]:
result_path = f"assign_results/results_{city}/Assignments_{city}.pickle" 

assignment = get_assignment(result_path, driver_locs, 7) 

temp_df = copy.deepcopy(driver_idf) 
# original base zones
temp_df = pd.merge(temp_df, base_zones_dict[city][['de_id', 'base_zone']], on='de_id')
temp_df['bz_idx'] = temp_df['base_zone'].map(zone_id2idx)
# base zones assigned by Fair Assign
temp_df['fa_bz_idx'] = assignment

**Random**

In [ ]:
# Random assignment while maintaining only upper capacity bounds of the zones:
def random_dist(day_num, driver_locs, upper_caps):
    random.seed(1234+day_num)
    num_drivers = len(driver_locs)
    num_zones = len(lower_caps)
    # print(num_drivers, num_zones)
    rand_df = pd.DataFrame(driver_locs, columns=["lat", "lng"])
    rand_df['bz_idx_rand'] = -1
    temp_upper_cap = copy.deepcopy(list(upper_caps))
    for i in range(num_drivers):
        zone = random.randint(1, num_zones)-1
        while(temp_upper_cap[zone]<=0):
            zone = random.randint(1, num_zones)-1
        rand_df.at[i, 'bz_idx_rand'] = zone 
        temp_upper_cap[zone] -= 1
    return rand_df

In [ ]:
rand_df = random_dist(driver_locs, upper_caps)
temp_df = copy.deepcopy(driver_idf) 
# original base zones
temp_df = pd.merge(temp_df, base_zones_dict[city][['de_id', 'base_zone']], on='de_id')
temp_df['bz_idx'] = temp_df['base_zone'].map(zone_id2idx)
# base zones assigned by Fair Assign
temp_df['fa_bz_idx'] = rand_df['bz_idx_rand']

**Round Robin**

In [ ]:
# RoundRobin assignment while maintaining upper capacity bounds of zones only:
def round_robin_dist(day_num, driver_locs):
    num_drivers = len(driver_locs)
    num_zones = len(lower_caps)
    # print(num_drivers, num_zones)
    rr_df = pd.DataFrame(driver_locs, columns=['lat', 'lng'])
    rr_df['bz_idx_rr'] = -1
    temp_upper_cap = copy.deepcopy(list(upper_caps))
    for i in range(num_drivers):
        zone = (day_num+1) % num_zones
        while(temp_upper_cap[zone]<=0):
            zone = (zone+1) % num_zones 
        rr_df.at[i, 'bz_idx_rr'] = int(zone)
        temp_upper_cap[zone] -= 1 
    return rr_df

In [ ]:
rr_df = round_robin_dist(day_num, driver_locs)
temp_df = copy.deepcopy(driver_idf) 
# original base zones
temp_df = pd.merge(temp_df, base_zones_dict[city][['de_id', 'base_zone']], on='de_id')
temp_df['bz_idx'] = temp_df['base_zone'].map(zone_id2idx)
# base zones assigned by Fair Assign
temp_df['fa_bz_idx'] = rr_df['bz_idx_rr']
# temp_df = temp_df.sort_values('de_id').reset_index()

**LIPA**

Use the first day of FoodMatch as the first day of LIPA

In [ ]:
# Requires simulation results of previous day
local_incomes_df = pd.DataFrame(columns=['de_id', 'day1', 'day2', 'day3', 'day4', 'day5', 'day6']) 
local_incomes_df['de_id'] = driver_idf['de_id']

day_incomes = {d_id:None for d_id in driver_idf['de_id']}

pre = 'A'
sim_path = f'sim_results/sim_results_{city}/sim.results{pre}lipa{day_num}'
print(sim_path)

data = pd.read_csv(sim_path, names=["a", "b", "c", "d", "e", "f", "g", "h"], on_bad_lines='skip')
data_deliver = data[data['a'] == "DELIVER"].drop(['a', 'e', 'f', 'g', 'h'], axis = 1)
data_deliver.columns = ['order_id', 'delivered_time', 'vehicle_id'] 
vehicle_ids = data_deliver['vehicle_id'].unique() 
    
data_deliver_gb = data_deliver.groupby('vehicle_id')
for d_id in driver_idf['de_id']:
    try:
        day_incomes[d_id] = int(data_deliver_gb.get_group(d_id).shape[0])
    except:
        # handles the cases for which d_id is not present in data_deliver 
        continue
                
local_incomes_df[f'day{day_num}'] = local_incomes_df['de_id'].map(day_incomes)

In [ ]:
# previous day incomes are used to determine the next day's assignment:
fm_incomes_df = pd.DataFrame(columns=['de_id', 'day1', 'day2', 'day3', 'day4', 'day5', 'day6']) 
fm_incomes_df['de_id'] = driver_idf['de_id']

fm_inc_df = copy.deepcopy(local_incomes_df)
fm_inc_df = pd.merge(fm_incomes_df, driver_idf, on='de_id')

# find day 1 incomes of drivers (FairAssign):
prev_incomes_df = copy.deepcopy(fm_inc_df[['de_id', 'lat', 'lng', 'day1']])
driver_prev_incomes = prev_incomes_df['day1'].values

# find day 1 number of orders in each zone (FairAssign):
orders_data = pd.read_csv(f"data/orders_data/{city}/orders_0{day_num}05.csv") 
sim_path = 'sim_results/sim_results_'+city+'/sim.resultsAlipa'+str(day_num)
   
data = pd.read_csv(sim_path, names=["a", "b", "c", "d", "e", "f", "g", "h"], on_bad_lines='skip')
data_deliver = data[data['a'] == "DELIVER"].drop(['a', 'e', 'f', 'g', 'h'], axis = 1)
data_deliver.columns = ['order_id', 'delivered_time', 'vehicle_id'] 
    
vehicle_ids = data_deliver['vehicle_id'].unique() 

df = pd.merge(data_deliver, orders_data, on='order_id') 

cust_zones = df['customer_zone'].unique()
cust_zones_gb = df.groupby('customer_zone')

orders_per_zone = {key:0 for key in cust_zones if key in zone_ids.values}
for key in cust_zones:
    if key in zone_ids.values:
        orders_per_zone[key] = cust_zones_gb.get_group(key).shape[0] 

# orders_per_zone = {k: v for k, v in sorted(orders_per_zone.items(), key=lambda item: item[1])}
orders_per_zone = {k:v for k, v in sorted(orders_per_zone.items())}
zone_prev_incomes = [v for k, v in orders_per_zone.items()]

In [ ]:
# LIPA while maintaining upper capacity bounds of zones only:
def lipa_dist(driver_locs, driver_prev_incomes, zone_prev_incomes):
    num_drivers = len(driver_locs)
    num_zones = len(lower_caps)

    lipa_df = pd.DataFrame(driver_locs, columns=["lat", "lng"])
    lipa_df['bz_idx_lipa'] = -1

    temp_upper_cap = copy.deepcopy(upper_caps)

    driver_idx_inc = np.argsort(np.array(driver_prev_incomes))
    zone_idx_inc = np.argsort(np.array(zone_prev_incomes))
  
    j = num_zones-1
    for i in driver_idx_inc:
        zone = zone_idx_inc[j]
        while(temp_upper_cap[zone]<=0):
            j = j-1
            zone = zone_idx_inc[j]
        lipa_df.at[i, 'bz_idx_lipa'] = zone
        temp_upper_cap[zone] -= 1
    return lipa_df

In [ ]:
lipa_df = lipa_dist(driver_locs, driver_prev_incomes, zone_prev_incomes)
temp_df = copy.deepcopy(driver_idf) 
# original base zones
temp_df = pd.merge(temp_df, base_zones_dict[city][['de_id', 'base_zone']], on='de_id')
temp_df['bz_idx'] = temp_df['base_zone'].map(zone_id2idx)
# base zones assigned by Fair Assign
temp_df['fa_bz_idx'] = lipa_df['bz_idx_lipa']

---

Creating modified files for current day 'day_num'

In [ ]:
# get the files corresponding to driver_idf:
de_idf_ids = driver_idf['de_id']

de_idf_files = []
for d_id in de_idf_ids:
    file_name = 'data/de_data/'+city +'_de_data/'+str(day_num) + '/de_intervals/' + str(int(d_id)) + '.csv'
    de_idf_files.append(file_name)
# de_idf_files

In [ ]:
# get the node_ids:
orig_node_ids = []
na = 0 # number of files in de_idf_files which are not present in de_intervals/
for file in de_idf_files:
    try:
        file_df = pd.read_csv(file)
    except:
        na += 1
        d_id = int(file.split('/')[-1][:-3])
        to_drop_idx = temp_df[temp_df['de_id']==d_id].index
        temp_df = temp_df.drop(to_drop_idx)
        print(to_drop_idx)
        continue 
    # get starting node ids corresponding to all shifts, it will be useful for random generation for unswappable drivers
    num_shifts = int(file_df.shape[0]/2)
    node_id = [int(file_df.iloc[x*2].values[0].split()[1]) for x in range(num_shifts)]
    orig_node_ids.append(node_id)

temp_df["node_id"] = orig_node_ids 
temp_df["fa_node_id"] = temp_df['node_id']

print(f"{na} intersection drivers not found in ../{day_num}/de_intervals/")
temp_df

In [ ]:
intersection_files = []
for d_id in driver_idf['de_id']:
    file_name = 'data/de_data/'+ city + '_de_data/'+ str(day_num) + '/de_intervals/' + str(int(d_id)) + '.csv'
    intersection_files.append(file_name)

old_dir_path = f'data/de_data/{city}_de_data/{day_num}/de_int_old'
new_dir_path = f'data/de_data/{city}_de_data/{day_num}/de_int_new'

if os.path.exists(old_dir_path):
    shutil.rmtree(old_dir_path)
    
if os.path.exists(new_dir_path):
    shutil.rmtree(new_dir_path)

os.mkdir(old_dir_path)
os.mkdir(new_dir_path)

for file in intersection_files:
    d_id = file[40:-4]
    try:
        file_df = pd.read_csv(file)
    except:
        continue 
    old_path = f'Code/data/de_data/{city}_de_data/{day_num}/de_int_old/{d_id}.csv'
    file_df.to_csv(old_path, index=False)
    new_path = f'data/de_data/{city}_de_data/{day_num}/de_int_new/{d_id}.csv'
    file_df.to_csv(new_path, index=False)

In [ ]:
# SWAPPING LOGIC:
# The swapping logic is applicable only to the drivers for which the fa_bz_idx is different from bz_idx
rel_df = temp_df[temp_df['bz_idx'] != temp_df['fa_bz_idx']].reset_index()
print(rel_df.shape[0])
rel_df['paired_de'+str(day_num)] = None

# first assign swappable nodes:
bz_nodes_dict = {zone_idx:{'freq':0, 'nodes':[], 'paired_de_id':[]} for zone_idx, _ in enumerate(zone_ids)} # it'll contain the frequency of each base zone in rel_df['bz_idx'] as well as the corresponding node_ids in a list
for idx in range(rel_df.shape[0]):
    z_id = int(rel_df.iloc[idx]['bz_idx'])
    n_id = int(rel_df.iloc[idx]['node_id'][0])
    paired_did = int(rel_df.iloc[idx]['de_id']) # remove later?
    bz_nodes_dict[z_id]['freq'] += 1
    bz_nodes_dict[z_id]['nodes'].append(n_id)
    bz_nodes_dict[z_id]['paired_de_id'].append(paired_did) # remove later?
bz_nodes_dict_store = copy.deepcopy(bz_nodes_dict)
# IF a required zone_id in fa_bz_idx is present in bz_nodes_dict then use that data point
# ELSE generate a random location in the zone corresponding to the zone_id
num_random = 0 # number of drivers for whom random generation of location was done to get the corresponding start node
for idx in range(rel_df.shape[0]):
    print(idx, end=' ')
    z_id = int(rel_df.iloc[idx]['fa_bz_idx'])
    if(bz_nodes_dict[z_id]['freq'] > 0):
        rel_df.loc[idx, 'fa_node_id'] = bz_nodes_dict[z_id]['nodes'][0]
        rel_df.loc[idx, 'paired_de'] = bz_nodes_dict[z_id]['paired_de_id'][0]
        # print(bz_nodes_dict[z_id]['paired_de_id'][0])
        bz_nodes_dict[z_id]['freq'] -= 1
        bz_nodes_dict[z_id]['nodes'].pop(0)
        bz_nodes_dict[z_id]['paired_de_id'].pop(0) # remove later?
    else:
        # randomly generate a location in the zone assigned by FairAssign
        zone_df = zone_dfs_dict[city]
        # zone_bdry = zone_df[zone_df['zone_id']==z_id]['path'].values[0] # Wrong ! bcz z_id is the index of zone_id
        zone_bdry = zone_df.iloc[z_id]['path']
        new_loc = generate_locs(1, zone_bdry)[0] 
        # shift new_loc: convert to anonymized coordinates
        if city=='ahm':
            new_loc[0] -= 2.0
            new_loc[1] -= 10.0
        if city=='blr':
            new_loc[0] += 3.0
            new_loc[1] -= 13.0
        if city=='del':
            new_loc[0] -= 12.0
            new_loc[1] -= 13.0
        # based on new_loc, get the closest node_id from location_df
        min_dist = 1e9
        n_id = -1
        for i in range(location_df.shape[0]):
            node_loc = [ location_df.iloc[i]['lat'], location_df.iloc[i]['lng'] ]
            curr_dist = euclidean_distance(new_loc, node_loc)
            if(curr_dist <= min_dist):
                min_dist = curr_dist 
                n_id = location_df.iloc[i]['node_id']     
        num_random += 1
        num_shifts = int(len(rel_df.iloc[idx]['node_id']))
        n_idz = [int(n_id)]*num_shifts
        rel_df.at[idx, 'fa_node_id'] = n_idz
print()
print(f"{rel_df.shape[0]-num_random} data points out of {rel_df.shape[0]} could be swapped !")
print(f"{num_random} data points were randomly generated !")

In [ ]:
intersection_files = []
for d_id in temp_df['de_id']:
    file_name = 'data/de_data/'+city + '_de_data/'+ str(day_num) + '/de_intervals/' + str(int(d_id)) + '.csv'
    intersection_files.append(file_name)

for file in intersection_files:
    # d_id = file[40:-4]
    d_id = int(file.split('/')[-1][:-4])
    file_df = pd.read_csv(file)

    old_path = f'data/de_data/{city}_de_data/{day_num}/de_int_old/{d_id}.csv'
    file_df.to_csv(old_path, index=False)

    new_path = f'Code/data/de_data/{city}_de_data/{day_num}/de_int_new/{d_id}.csv'
    file_df.to_csv(new_path, index=False)
    

In [ ]:
# the de_interval profiles of swappable or paired drivers are to be swapped 
# and that of drivers for whom random locations are generated, the profile is to be modified
rel_drivers = rel_df['de_id']

rel_files = []
for d_id in rel_drivers:
    file_name = 'data/de_data/'+city + '_de_data/'+ str(day_num) + '/de_intervals/' + str(int(d_id)) + '.csv'
    rel_files.append(file_name)

for file in rel_files:
    # d_id = file[40:-4] 
    d_id = int(file.split('/')[-1][:-4])
    file_df = pd.read_csv(file)
    
    swap_node = rel_df[rel_df['de_id']==int(d_id)].paired_de.values[0]
    # for those drivers who could be swapped:
    if not np.isnan(swap_node):
        swap_with_file = 'data/de_data/' + city + '_de_data/'+ str(day_num) + '/de_intervals/' + str(int(swap_node)) + '.csv'
        file_df = pd.read_csv(swap_with_file)
    # for those drivers whose starting nodes for each shift were randomly generated
    else:
        num_shifts = int(file_df.shape[0]/2)
        new_start_nodes = rel_df[rel_df['de_id']==int(d_id)].fa_node_id.values[0]
        for i in range(num_shifts):
            new_node = new_start_nodes[i]
            file_df.iloc[i*2] = str(file_df.iloc[0].values[0].split()[0])+ ' ' + str(new_node) 

    new_path = f'data/de_data/{city}_de_data/{day_num}/de_int_new/{d_id}.csv'
    file_df.to_csv(new_path, index=False)

---

**Evaluation**

In [ ]:
# METRICS:

def gini_index(incomes):
    num = len(incomes)
    total = incomes.sum() 
    inc_sum = 0.0
    for i in range(num):
        for j in range(num):
            inc_sum += abs(incomes[i]-incomes[j])
    gini = inc_sum / (2*num*total)
    return gini


def avg_distance(zone_labels, driver_locs, zone_locs):
    """
    returns the 'cost' of the assignment
    zone_labels: indices of the assigned zones
    a zone_label 'z' has location zone_locs[z]
    """
    driver_dists = L2Distance(driver_locs) 
    num = len(zone_labels)
    dist = 0.0 
    for i in range(num):
        assigned_zone = zone_labels[i]
        driver_loc, zone_loc = driver_locs[i], zone_locs[int(assigned_zone)]
        driver_zone_dist = euclidean_distance(driver_loc, zone_loc)
        dist += np.sqrt(driver_zone_dist)
    avg_dist = dist/num
    return avg_dist                       
    

def spatial_inequality_index(incomes, driver_locs, ratings, combined, fair_distance):
    if flag==0:
        driver_dists = L2Distance(driver_locs)
    if flag==1:
        driver_dists = abs_difference(ratings)
    if flag==2:
        driver_dists = combined
    num = len(incomes)
    total = incomes.sum()
    term_i = 0.0    
    for i in range(num):
        sum_j = 0.0
        num_j = 1e-9    
        for j in range(i+1, num):
            if driver_dists[i][j] <= fair_distance and driver_dists[i][j]>0:
                num_j += 1
                sum_j += abs(incomes[i]-incomes[j])   
        term_i += (sum_j / num_j) 
    
    spin_idx = term_i / total 
    # spin_idx = round(spin_idx, 2)
    return spin_idx 


def income_gap(incomes, driver_locs, ratings, combined, fair_distance):
    """ 
    difference between incomes between any two drivers per unit distance (within fair_distance) 
    """
    alpha = 100

    if flag==0:
        driver_dists = L2Distance(driver_locs)
    if flag==1:
        driver_dists = abs_difference(ratings)
    if flag==2:
        driver_dists = combined
    driver_dists = driver_dists * alpha
    num = len(incomes)
    total = incomes.sum()
    terms = 0.0
    num_pair_drivers = 1e-7 # NOT 0 => to avoid division by 0
    for i in range(num-1):
        for j in range(i+1, num):
            if driver_dists[i][j]>0:
                num_pair_drivers += 1
                terms += (abs(incomes[i]-incomes[j])/driver_dists[i][j])
    inc_gap = terms/num_pair_drivers
    # inc_gap = round(inc_gap, 2)
    return inc_gap

After getting the FoodMatch simulation results for all 6 days

In [ ]:
def get_incomes_df(algo):
    # get the incomes on all 6 days for all drivers 
    '''
    input string: algo
    'fm' : FoodMatch
    'fafm' : FairAssign + FoodMatch
    '''
    local_incomes_df = pd.DataFrame(columns=['de_id', 'day1', 'day2', 'day3', 'day4', 'day5', 'day6']) 
    local_incomes_df['de_id'] = driver_idf['de_id']

    num_days = 6
    day_incomes = {d_id:None for d_id in driver_idf['de_id']}

    pre = 'A'
    for day in range(1, num_days+1):
        sim_path = f'sim_results/sim_results_{city}/sim.results{pre}{algo}{day}'
        print(sim_path)
        data = pd.read_csv(sim_path, names=["a", "b", "c", "d", "e", "f", "g", "h"], on_bad_lines='skip')
        data_deliver = data[data['a'] == "DELIVER"].drop(['a', 'e', 'f', 'g', 'h'], axis = 1)
        # print(data_deliver)
        data_deliver.columns = ['order_id', 'delivered_time', 'vehicle_id'] 
        vehicle_ids = data_deliver['vehicle_id'].unique() 
        data_deliver_gb = data_deliver.groupby('vehicle_id')
        for d_id in driver_idf['de_id']:
            try:
                day_incomes[d_id] = int(data_deliver_gb.get_group(d_id).shape[0])
            except:
                # handles the cases for which d_id is not present in data_deliver 
                continue  
        local_incomes_df[f'day{day}'] = local_incomes_df['de_id'].map(day_incomes)
    
    cols = ['day1', 'day2', 'day3', 'day4', 'day5', 'day6'] 
    local_incomes_df['num_orders'] = local_incomes_df[cols].sum(axis=1) 

    return copy.deepcopy(local_incomes_df)

In [ ]:
# Only FoodMatch: 
fm_incomes_df = pd.DataFrame(columns=['de_id', 'day1', 'day2', 'day3', 'day4', 'day5', 'day6']) 
fm_incomes_df['de_id'] = driver_idf['de_id']
fm_incomes_df = get_incomes_df('fm')
fm_incomes_df = pd.merge(fm_incomes_df, driver_idf, on='de_id')

# FairAssign then FoodMatch:
fafm_incomes_df = pd.DataFrame(columns=['de_id', 'day1', 'day2', 'day3', 'day4', 'day5', 'day6']) 
fafm_incomes_df['de_id'] = driver_idf['de_id']
fafm_incomes_df = get_incomes_df('fafm')
fafm_incomes_df = pd.merge(fafm_incomes_df, driver_idf, on='de_id')

fm_inc_df = copy.deepcopy(fm_incomes_df)
fafm_inc_df = copy.deepcopy(fafm_incomes_df)
fm_inc_df = pd.merge(fm_incomes_df, fafm_incomes_df[['de_id']])
fafm_inc_df = pd.merge(fafm_incomes_df, fm_inc_df[['de_id']])
assert fm_inc_df.shape[0]==fafm_inc_df.shape[0]

Calculate metrics

In [ ]:
lats_fm = fm_inc_df['lat']
longs_fm = fm_inc_df['lng']

lats_fafm = fafm_inc_df['lat']
longs_fafm = fafm_inc_df['lng'] 

fm_incomes = fm_inc_df['num_orders']
fafm_incomes = fafm_inc_df['num_orders']

In [ ]:
def metrics(lats, longs, incomes, ratings, combined, fair_dist): 
    d_locs = [[lat, lng] for lat, lng in zip(lats, longs)]
    incomes = np.array(incomes)
    gini = gini_index(incomes) 
    sp_idx = spatial_inequality_index(incomes, d_locs, ratings, combined, fair_dist)
    inc_gp = income_gap(incomes, d_locs, ratings, combined, fair_dist)
    return gini, sp_idx, inc_gp

In [ ]:
if flag==0:
    driver_dists = driver_dists 
if flag==1:
    driver_dists = ratings_matrix 
if flag==2:
    driver_dists = combined

def num_sim_drivers(fd):
    ''' 
    number of similar drivers (i.e., for a given driver, how many drivers are being considered for fairness comparison per)
    '''
    num_similar_drivers = []
    for idx in range(len(driver_dists)):
        curr_driver = driver_dists[idx]
        num_sim = curr_driver[curr_driver<=fd].shape[0]
        num_similar_drivers.append(num_sim) 
    return np.mean(num_similar_drivers)

In [ ]:
# calculating the metrics at different fair_dist values: 
if flag==0:
    fair_dist = driver_dists.mean()/8 # actual: cd 
if flag==1:
    fair_dist = 1 # 0.5
if flag==2:
    fair_dist = 0.04 # actual: 0.04
    
def eval_results(lats, longs, incomes):
    results = []
    for k in range(1, 11):
        fd = fair_dist/k 
        gini, sp_idx, inc_gp = metrics(lats, longs, incomes, ratings, combined, fd)
        results.append([fd, num_sim_drivers(fd), gini, sp_idx, inc_gp])
    result_df = pd.DataFrame(results)
    cols = ['fair_dist', 'sim_drivers', 'gini', 'spatial_ineq', 'income_gap']
    result_df.columns = cols
    return result_df

# FINAL RESULTS:
fm_results_df = eval_results(lats_fm, longs_fm, fm_incomes) 
fafm_results_df = eval_results(lats_fafm, longs_fafm, fafm_incomes)

Calculating Avg. Distance (or Cost)

In [ ]:
def distance_this_driver(locs, v_id):
    # driver's inital location:
    v_init_loc = driver_idf[driver_idf['de_id']==v_id][['lat', 'lng']].values[0]
    first_mile_dist = euclidean_distance(locs[0], v_init_loc)
    last_mile_dist = 0
    for idx in range(1, len(locs)):
        prev_loc = locs[idx-1]
        curr_loc = locs[idx] 
        last_mile_dist += euclidean_distance(prev_loc, curr_loc) 
    return first_mile_dist, last_mile_dist

In [ ]:
# get the incomes on all 6 days for all drivers 
num_days = 6

def get_cost(algo, num_days):
    ''' 
    algo: str
    'fm': FoodMatch 
    'fafm': FairAssign then FoodMatch
    '''
    cust_lats, cust_lngs = [], []
    first_mile_cost = 0.0 # over all num_days days
    last_mile_cost = 0.0 # over all num_days days
    for day in range(1, num_days+1):
        total_first_mile = 0
        total_last_mile = 0
        orders_data = pd.read_csv(f"data/orders_data/{city}/orders_0{day}05.csv") 
        sim_path = f'sim_results/sim_results_{city}/sim.resultsA{algo}(day)'
        
        data = pd.read_csv(sim_path, names=["a", "b", "c", "d", "e", "f", "g", "h"], on_bad_lines='skip')
        data_deliver = data[data['a'] == "DELIVER"].drop(['a', 'e', 'f', 'g', 'h'], axis = 1)
        data_deliver.columns = ['order_id', 'delivered_time', 'vehicle_id'] 
        
        vehicle_ids = data_deliver['vehicle_id'].unique() 

        df = pd.merge(data_deliver, orders_data, on='order_id') 
        c_locs = df['customer_lat_lng'].values 
        df['cust_lat'] = [float(loc.split(',')[0]) for loc in c_locs]
        df['cust_lng'] = [float(loc.split(',')[1]) for loc in c_locs]

        cust_lats.append(df['cust_lat'])
        cust_lngs.append(df['cust_lng'])

        df_gb = df.groupby('vehicle_id') # Don't group on 'de_id' 
        for v_id in vehicle_ids:
            curr_group = df_gb.get_group(v_id)
            first_mile_dist, last_mile_dist = distance_this_driver(curr_group[['cust_lat', 'cust_lng']].values, v_id)
            total_first_mile += first_mile_dist 
            total_last_mile += last_mile_dist
        # avg cost for this day:
        first_mile_cost += (total_first_mile/len(vehicle_ids))
        last_mile_cost += (total_last_mile/len(vehicle_ids)) 
    # avg cost over all days:
    first_mile_cost = first_mile_cost/num_days
    last_mile_cost = last_mile_cost/num_days 
    cost = first_mile_cost + last_mile_cost 
    # print(first_mile_cost, last_mile_cost, cost)
    return first_mile_cost, last_mile_cost, cost

In [ ]:
# costs with only FoodMatch:
first_mile, last_mile, total = get_cost('fm', num_days)
print(total) 

# costs with FairAssign + FoodMatch: 
first_mile_, last_mile_, total_ = get_cost('fafm', num_days)
print(total)

---